# Filtro de Dragón que escupe fuego cuando se abre la boca


In [5]:
import cv2
import dlib
import numpy as np
import math

# Cargar el predictor de puntos faciales
predictor_path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Cargar la máscara de dragón y la imagen de fuego
mask = cv2.imread("dragon_mask_1.png", cv2.IMREAD_UNCHANGED)
fire_effect = cv2.imread("fire.png", cv2.IMREAD_UNCHANGED)

# Configuración inicial
fire_positions = []
fire_length_increment = 10
max_fire_length = 200
fire_emoji_size = (32, 32)

fire_angle = 0

def is_mouth_open(shape):
    top_lip = shape.part(51).y
    bottom_lip = shape.part(57).y
    lip_distance = bottom_lip - top_lip
    return lip_distance > 35

def overlay_mask(frame, face, shape, mask):
    # Obtener los puntos 8 (barbilla), 2 (izquierda) y 14 (derecha) de la cara
    chin = (shape.part(8).x, shape.part(8).y)
    left_side = (shape.part(2).x, shape.part(2).y)
    right_side = (shape.part(14).x, shape.part(14).y)

    # Calcular el centro de la cara y la distancia entre los lados izquierdo y derecho
    face_center = ((left_side[0] + right_side[0]) // 2, (left_side[1] + right_side[1]) // 2)
    face_width = int(np.linalg.norm(np.array(right_side) - np.array(left_side)))+150

    # Ajuste vertical para posicionar la máscara más cerca de la nariz
    mask_position_y = (chin[1] + face_center[1]) // 2 - 120

    # Escalar la máscara según la anchura de la cara
    scale_factor = face_width / mask.shape[1]
    mask_resized = cv2.resize(mask, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
    mask_rgb = mask_resized[:, :, :3]
    mask_alpha = mask_resized[:, :, 3] / 255.0

    # Rotar la máscara para que se alinee con el ángulo de la cara
    angle = math.degrees(math.atan2(right_side[1] - left_side[1], right_side[0] - left_side[0]))
    rotation_matrix = cv2.getRotationMatrix2D((mask_resized.shape[1] // 2, mask_resized.shape[0] // 2), angle, 1)
    mask_rgb = cv2.warpAffine(mask_rgb, rotation_matrix, (mask_resized.shape[1], mask_resized.shape[0]))
    mask_alpha = cv2.warpAffine(mask_alpha, rotation_matrix, (mask_resized.shape[1], mask_resized.shape[0]))

    # Ajustar la posición de la máscara en la cara
    x_offset = face_center[0] - mask_rgb.shape[1] // 2
    y_offset = mask_position_y - mask_rgb.shape[0] // 2

    y1, y2 = max(0, y_offset), min(y_offset + mask_rgb.shape[0], frame.shape[0])
    x1, x2 = max(0, x_offset), min(x_offset + mask_rgb.shape[1], frame.shape[1])

    mask_rgb = mask_rgb[:y2 - y1, :x2 - x1]
    mask_alpha = mask_alpha[:y2 - y1, :x2 - x1]
    roi = frame[y1:y2, x1:x2]

    for c in range(3):
        roi[:, :, c] = (mask_alpha * mask_rgb[:, :, c] + (1 - mask_alpha) * roi[:, :, c])

    frame[y1:y2, x1:x2] = roi
    return frame

def overlay_fire_sequence(frame, fire_positions, fire_effect):
    fire_resized = cv2.resize(fire_effect, fire_emoji_size, interpolation=cv2.INTER_AREA)
    fire_rgb = fire_resized[:, :, :3]
    fire_alpha = fire_resized[:, :, 3] / 255.0

    for (x, y) in fire_positions:
        h, w = fire_resized.shape[:2]
        y1, y2 = max(0, y), min(y + h, frame.shape[0])
        x1, x2 = max(0, x - w // 2), min(x - w // 2 + w, frame.shape[1])

        if y2 <= y1 or x2 <= x1:
            continue

        fire_rgb_resized = fire_rgb[:y2 - y1, :x2 - x1]
        fire_alpha_resized = fire_alpha[:y2 - y1, :x2 - x1]
        roi = frame[y1:y2, x1:x2]

        for c in range(3):
            roi[:, :, c] = (fire_alpha_resized * fire_rgb_resized[:, :, c] + (1 - fire_alpha_resized) * roi[:, :, c])

    return frame

# Inicializar captura de video
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    if len(faces) > 0:
        for face in faces:
            shape = predictor(gray, face)

            left_mouth = (shape.part(48).x, shape.part(48).y)
            right_mouth = (shape.part(54).x, shape.part(54).y)
            mouth_center = ((left_mouth[0] + right_mouth[0]) // 2, (left_mouth[1] + right_mouth[1]) // 2)

            dx = right_mouth[0] - left_mouth[0]
            dy = right_mouth[1] - left_mouth[1]
            magnitude = np.sqrt(dx**2 + dy**2)
            perp_vector = (-dy / magnitude, dx / magnitude)

            rad_angle = math.radians(fire_angle)
            rotated_vector = (
                perp_vector[0] * math.cos(rad_angle) - perp_vector[1] * math.sin(rad_angle),
                perp_vector[0] * math.sin(rad_angle) + perp_vector[1] * math.cos(rad_angle)
            )

            if is_mouth_open(shape):
                mouth_status = "Mouth: Open"
                new_fire_pos = (
                    int(mouth_center[0] + rotated_vector[0] * len(fire_positions) * fire_length_increment),
                    int(mouth_center[1] + rotated_vector[1] * len(fire_positions) * fire_length_increment)
                )
                fire_positions.append(new_fire_pos)

                if len(fire_positions) * fire_length_increment > max_fire_length:
                    fire_positions.pop(0)
            else:
                mouth_status = "Mouth: Closed"
                fire_positions.clear()

            frame = overlay_mask(frame, face, shape, mask)
            frame = overlay_fire_sequence(frame, fire_positions, fire_effect)

    else:
        mouth_status = "Mouth: Not detected"
        fire_positions.clear()

    
    
    cv2.imshow("Face with Dragon Mask and Fire Effect", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('a'):
        fire_angle -= 5
    elif key == ord('d'):
        fire_angle += 5
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
